In [75]:
# Import necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
import statsmodels.api as sum
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [76]:
df = pd.read_csv('us2022q2a.csv')
df

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,totalliabilities,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.761670,104.0000,452000.000,NaN,2000.0,1.0
1,A,2000q2,2485000.0,1261000.0,1.010000e+06,0.0,42000.000000,0.000,90000.0,7321000.000,2679000.000,512000.000,0.000,4642000.000,45.215607,73.7500,452271.967,6.0,2000.0,2.0
2,A,2000q3,2670000.0,1369000.0,1.091000e+06,0.0,28000.000000,0.000,83000.0,7827000.000,2925000.000,528000.000,0.000,4902000.000,30.003238,48.9375,453014.579,9.0,2000.0,3.0
3,A,2000q4,3372000.0,1732000.0,1.182000e+06,0.0,10000.000000,0.000,163000.0,8425000.000,3160000.000,830000.000,0.000,5265000.000,33.566841,54.7500,456366.381,12.0,2000.0,4.0
4,A,2001q1,2841000.0,1449000.0,1.113000e+06,0.0,-6000.000000,0.000,119000.0,9208000.000,3667000.000,556000.000,0.000,5541000.000,18.840347,30.7300,456769.737,3.0,2001.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324442,ZYNE,2021q3,0.0,0.0,1.021065e+04,0.0,-376.636750,-5.038,0.0,89996.170,11309.171,207.635,411.237,78686.999,4.240000,4.2400,41251.537,9.0,2021.0,3.0
324443,ZYNE,2021q4,0.0,0.0,8.836436e+03,0.0,16.937906,-4.433,0.0,81171.507,10258.173,209.068,353.694,70913.334,2.880000,2.8800,41217.537,12.0,2021.0,4.0
324444,ZYNE,2022q1,0.0,0.0,8.903915e+03,0.0,317.252110,-96.044,0.0,74381.029,9214.059,210.512,295.754,65166.970,2.050000,2.0500,42447.037,3.0,2022.0,1.0
324445,ZYNE,2022q2,0.0,0.0,9.168770e+03,0.0,-775.927860,-91.691,0.0,67006.959,9765.086,211.965,237.414,57241.873,1.140000,1.1400,43595.959,6.0,2022.0,2.0


In [77]:
# Define name of columns and delete the ones that we wont be using
columns = ["firm", "Name", "N", "Class", "Country of Origin", "Type of Asset", "Sector NAICS level 1", "Exchange / Src", "Sector Economatica", "Sector NAICS last available", "partind"]
firms = pd.read_csv("usfirms2022.csv", names=columns)
firms = firms.drop(["N", "Name","Class", "Country of Origin", "Type of Asset", "Exchange / Src", "Sector Economatica", "Sector NAICS last available", "partind"], axis = 1)

In [78]:
# Merge data sets to get all the information we need
df = pd.merge(df,firms, on = 'firm')
df

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto,Sector NAICS level 1
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,63.761670,104.0000,452000.000,NaN,2000.0,1.0,Manufacturing
1,A,2000q2,2485000.0,1261000.0,1.010000e+06,0.0,42000.000000,0.000,90000.0,7321000.000,...,512000.000,0.000,4642000.000,45.215607,73.7500,452271.967,6.0,2000.0,2.0,Manufacturing
2,A,2000q3,2670000.0,1369000.0,1.091000e+06,0.0,28000.000000,0.000,83000.0,7827000.000,...,528000.000,0.000,4902000.000,30.003238,48.9375,453014.579,9.0,2000.0,3.0,Manufacturing
3,A,2000q4,3372000.0,1732000.0,1.182000e+06,0.0,10000.000000,0.000,163000.0,8425000.000,...,830000.000,0.000,5265000.000,33.566841,54.7500,456366.381,12.0,2000.0,4.0,Manufacturing
4,A,2001q1,2841000.0,1449000.0,1.113000e+06,0.0,-6000.000000,0.000,119000.0,9208000.000,...,556000.000,0.000,5541000.000,18.840347,30.7300,456769.737,3.0,2001.0,1.0,Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323811,ZYNE,2021q2,0.0,0.0,9.838494e+03,0.0,-117.528220,-5.943,0.0,98195.904,...,206.211,468.385,87699.257,5.290000,5.2900,41251.537,6.0,2021.0,2.0,Manufacturing
323812,ZYNE,2021q3,0.0,0.0,1.021065e+04,0.0,-376.636750,-5.038,0.0,89996.170,...,207.635,411.237,78686.999,4.240000,4.2400,41251.537,9.0,2021.0,3.0,Manufacturing
323813,ZYNE,2021q4,0.0,0.0,8.836436e+03,0.0,16.937906,-4.433,0.0,81171.507,...,209.068,353.694,70913.334,2.880000,2.8800,41217.537,12.0,2021.0,4.0,Manufacturing
323814,ZYNE,2022q1,0.0,0.0,8.903915e+03,0.0,317.252110,-96.044,0.0,74381.029,...,210.512,295.754,65166.970,2.050000,2.0500,42447.037,3.0,2022.0,1.0,Manufacturing


In [79]:
# Stablish firm as index with quartil frequency
df.set_index(['firm', 'q'], inplace=True)
df

revenue       cogs          sgae  otheropexp   extraincome  \
firm q                                                                      
A    2000q1        NaN        NaN           NaN         NaN           NaN   
     2000q2  2485000.0  1261000.0  1.010000e+06         0.0  42000.000000   
     2000q3  2670000.0  1369000.0  1.091000e+06         0.0  28000.000000   
     2000q4  3372000.0  1732000.0  1.182000e+06         0.0  10000.000000   
     2001q1  2841000.0  1449000.0  1.113000e+06         0.0  -6000.000000   
...                ...        ...           ...         ...           ...   
ZYNE 2021q2        0.0        0.0  9.838494e+03         0.0   -117.528220   
     2021q3        0.0        0.0  1.021065e+04         0.0   -376.636750   
     2021q4        0.0        0.0  8.836436e+03         0.0     16.937906   
     2022q1        0.0        0.0  8.903915e+03         0.0    317.252110   
     2022q2        0.0        0.0  9.168770e+03         0.0   -775.927860   

             finexp  incometax  totalassets  totalliabilities   shortdebt  \
firm q                                                                      
A    2000q1     NaN        NaN          NaN               NaN         NaN   
     2000q2   0.000    90000.0  7321000.000       2679000.000  512000.000   
     2000q3   0.000    83000.0  7827000.000       2925000.000  528000.000   
     2000q4   0.000   163000.0  8425000.000       3160000.000  830000.000   
     2001q1   0.000   119000.0  9208000.000       3667000.000  556000.000   
...             ...        ...          ...               ...         ...   
ZYNE 2021q2  -5.943        0.0    98195.904         10496.647     206.211   
     2021q3  -5.038        0.0    89996.170         11309.171     207.635   
     2021q4  -4.433        0.0    81171.507         10258.173     209.068   
     2022q1 -96.044        0.0    74381.029          9214.059     210.512   
     2022q2 -91.691        0.0    67006.959          9765.086     211.965   

             longdebt  stockholderequity   adjprice  originalprice  \
firm q                                                               
A    2000q1       NaN                NaN  63.761670       104.0000   
     2000q2     0.000        4642000.000  45.215607        73.7500   
     2000q3     0.000        4902000.000  30.003238        48.9375   
     2000q4     0.000        5265000.000  33.566841        54.7500   
     2001q1     0.000        5541000.000  18.840347        30.7300   
...               ...                ...        ...            ...   
ZYNE 2021q2   468.385          87699.257   5.290000         5.2900   
     2021q3   411.237          78686.999   4.240000         4.2400   
     2021q4   353.694          70913.334   2.880000         2.8800   
     2022q1   295.754          65166.970   2.050000         2.0500   
     2022q2   237.414          57241.873   1.140000         1.1400   

             sharesoutstanding  fiscalmonth    year  cto Sector NAICS level 1  
firm q                                                                         
A    2000q1         452000.000          NaN  2000.0  1.0        Manufacturing  
     2000q2         452271.967          6.0  2000.0  2.0        Manufacturing  
     2000q3         453014.579          9.0  2000.0  3.0        Manufacturing  
     2000q4         456366.381         12.0  2000.0  4.0        Manufacturing  
     2001q1         456769.737          3.0  2001.0  1.0        Manufacturing  
...                        ...          ...     ...  ...                  ...  
ZYNE 2021q2          41251.537          6.0  2021.0  2.0        Manufacturing  
     2021q3          41251.537          9.0  2021.0  3.0        Manufacturing  
     2021q4          41217.537         12.0  2021.0  4.0        Manufacturing  
     2022q1          42447.037          3.0  2022.0  1.0        Manufacturing  
     2022q2          43595.959          6.0  2022.0  2.0        Manufacturing  

[323816 rows x 19 columns]

In [80]:
df["Market Value"] = df["originalprice"] * df["sharesoutstanding"]
df[["Market Value"]]

Market Value
firm q                   
A    2000q1  4.700800e+07
     2000q2  3.335506e+07
     2000q3  2.216940e+07
     2000q4  2.498606e+07
     2001q1  1.403653e+07
...                   ...
ZYNE 2021q2  2.182206e+05
     2021q3  1.749065e+05
     2021q4  1.187065e+05
     2022q1  8.701643e+04
     2022q2  4.969939e+04

[323816 rows x 1 columns]

In [81]:
# Get Book Value
df["Book Value"] = df["totalassets"] - df["totalliabilities"]
df[["Book Value"]]

Book Value
firm q                  
A    2000q1          NaN
     2000q2  4642000.000
     2000q3  4902000.000
     2000q4  5265000.000
     2001q1  5541000.000
...                  ...
ZYNE 2021q2    87699.257
     2021q3    78686.999
     2021q4    70913.334
     2022q1    65166.970
     2022q2    57241.873

[323816 rows x 1 columns]

In [82]:
# Get Earning Before Interests and Taxes (Ebit)
df["Ebit"] = df["revenue"] - df["cogs"] - df["sgae"]
df[["Ebit"]]

Ebit
firm q                  
A    2000q1          NaN
     2000q2  214000.0000
     2000q3  210000.0000
     2000q4  458000.0000
     2001q1  279000.0000
...                  ...
ZYNE 2021q2   -9838.4941
     2021q3  -10210.6520
     2021q4   -8836.4365
     2022q1   -8903.9150
     2022q2   -9168.7695

[323816 rows x 1 columns]

In [83]:
df["OPM"] = (df["Ebit"]/df["revenue"]).replace([np.inf, -np.inf], np.nan)
df[["OPM"]]

OPM
firm q               
A    2000q1       NaN
     2000q2  0.086117
     2000q3  0.078652
     2000q4  0.135824
     2001q1  0.098205
...               ...
ZYNE 2021q2       NaN
     2021q3       NaN
     2021q4       NaN
     2022q1       NaN
     2022q2       NaN

[323816 rows x 1 columns]

In [84]:
# Get Net Income
df["Net Income"] = df["Ebit"] - df["incometax"] - df["finexp"] + df["extraincome"]
df[["Net Income"]]

Net Income
firm q                    
A    2000q1            NaN
     2000q2  166000.000000
     2000q3  155000.000000
     2000q4  305000.000000
     2001q1  154000.000000
...                    ...
ZYNE 2021q2   -9950.079320
     2021q3  -10582.250750
     2021q4   -8815.065594
     2022q1   -8490.618890
     2022q2   -9853.006360

[323816 rows x 1 columns]

In [85]:
# Get Prfit Margin
df["Profit Margin"] = ((df["Net Income"] / df["revenue"])*100).replace([np.inf, -np.inf], np.nan)
df[["Profit Margin"]]

Profit Margin
firm q                    
A    2000q1            NaN
     2000q2       6.680080
     2000q3       5.805243
     2000q4       9.045077
     2001q1       5.420627
...                    ...
ZYNE 2021q2            NaN
     2021q3            NaN
     2021q4            NaN
     2022q1            NaN
     2022q2            NaN

[323816 rows x 1 columns]

In [86]:
# Create earnings per share variable
EPS = df["Ebit"] / df["sharesoutstanding"]

In [87]:
# Create earnings per share deflated by price column
df["EPSP"] = EPS / df["originalprice"]
df[["EPSP"]]

EPSP
firm q               
A    2000q1       NaN
     2000q2  0.006416
     2000q3  0.009473
     2000q4  0.018330
     2001q1  0.019877
...               ...
ZYNE 2021q2 -0.045085
     2021q3 -0.058378
     2021q4 -0.074439
     2022q1 -0.102325
     2022q2 -0.184485

[323816 rows x 1 columns]

In [88]:
# Get sales annual growth
df["SAG"] = (df["revenue"] / (df["revenue"].shift(-4)))-1
df[["SAG"]]

SAG
firm q               
A    2000q1       NaN
     2000q2  0.043241
     2000q3  0.478405
     2000q4  1.031325
     2001q1  0.992286
...               ...
ZYNE 2021q2       NaN
     2021q3       NaN
     2021q4       NaN
     2022q1       NaN
     2022q2       NaN

[323816 rows x 1 columns]

In [89]:
# Get Operating Profit Growth
df["OPG"] = (df["Ebit"] / (df["Ebit"].shift(-4)))-1
df[["OPG"]]

OPG
firm q               
A    2000q1       NaN
     2000q2 -4.451613
     2000q3 -1.614035
     2000q4 -1.696049
     2001q1 -1.631222
...               ...
ZYNE 2021q2  0.073044
     2021q3       NaN
     2021q4       NaN
     2022q1       NaN
     2022q2       NaN

[323816 rows x 1 columns]

In [90]:
# Get Book to Market Ratio 
df["BMR"] = df["Book Value"] / df["Market Value"]
df[["BMR"]]

BMR
firm q               
A    2000q1       NaN
     2000q2  0.139169
     2000q3  0.221116
     2000q4  0.210718
     2001q1  0.394756
...               ...
ZYNE 2021q2  0.401883
     2021q3  0.449880
     2021q4  0.597384
     2022q1  0.748904
     2022q2  1.151762

[323816 rows x 1 columns]

In [91]:
# Get short financial leverage 
df["SFL"] = df["shortdebt"] / df["totalassets"]
df[["SFL"]]

SFL
firm q               
A    2000q1       NaN
     2000q2  0.069936
     2000q3  0.067459
     2000q4  0.098516
     2001q1  0.060382
...               ...
ZYNE 2021q2  0.002100
     2021q3  0.002307
     2021q4  0.002576
     2022q1  0.002830
     2022q2  0.003163

[323816 rows x 1 columns]

In [92]:
# Get long financial leverage
df["LFL"] = df["longdebt"] / df["totalassets"]
df[["LFL"]]

LFL
firm q               
A    2000q1       NaN
     2000q2  0.000000
     2000q3  0.000000
     2000q4  0.000000
     2001q1  0.000000
...               ...
ZYNE 2021q2  0.004770
     2021q3  0.004569
     2021q4  0.004357
     2022q1  0.003976
     2022q2  0.003543

[323816 rows x 1 columns]

In [93]:
# Get dependant variable - stock returns
df["r"] = np.log(df["adjprice"]) - np.log(df["adjprice"].shift(4))
df[["r"]]

r
firm q               
A    2000q1       NaN
     2000q2       NaN
     2000q3       NaN
     2000q4       NaN
     2001q1 -1.219152
...               ...
ZYNE 2021q2  0.450906
     2021q3  0.247615
     2021q4 -0.136132
     2022q1 -0.819027
     2022q2 -1.534790

[323816 rows x 1 columns]

In [94]:
df[["adjprice"]]

adjprice
firm q                
A    2000q1  63.761670
     2000q2  45.215607
     2000q3  30.003238
     2000q4  33.566841
     2001q1  18.840347
...                ...
ZYNE 2021q2   5.290000
     2021q3   4.240000
     2021q4   2.880000
     2022q1   2.050000
     2022q2   1.140000

[323816 rows x 1 columns]

In [95]:
df["F1r"] = df["r"].shift(-1)
df[["F1r"]]

F1r
firm q               
A    2000q1       NaN
     2000q2       NaN
     2000q3       NaN
     2000q4 -1.219152
     2001q1 -0.819441
...               ...
ZYNE 2021q2  0.247615
     2021q3 -0.136132
     2021q4 -0.819027
     2022q1 -1.534790
     2022q2       NaN

[323816 rows x 1 columns]

In [96]:
commercial = df.loc[(df["Sector NAICS level 1"] == "Wholesale Trade") | (df["Sector NAICS level 1"] == "Retail Trade")]
commercial

revenue      cogs     sgae  otheropexp  extraincome  finexp  \
firm q                                                                      
AAP  2000q1       NaN       NaN      NaN         NaN          NaN     NaN   
     2000q2       NaN       NaN      NaN         NaN          NaN     NaN   
     2000q3       NaN       NaN      NaN         NaN          NaN     NaN   
     2000q4       NaN       NaN      NaN         NaN          NaN     NaN   
     2001q1       NaN       NaN      NaN         NaN          NaN     NaN   
...               ...       ...      ...         ...          ...     ...   
ZUMZ 2021q2  279069.0  175900.0  68889.0         0.0        254.0  -975.0   
     2021q3  268666.0  163701.0  73011.0         0.0       -151.0  -965.0   
     2021q4  289455.0  174791.0  74822.0         0.0        468.0  -893.0   
     2022q1  346677.0  212744.0  82198.0         0.0      -1462.0  -759.0   
     2022q2  220686.0  148312.0  71877.0         0.0        172.0  -492.0   

             incometax  totalassets  totalliabilities  shortdebt  ...  \
firm q                                                            ...   
AAP  2000q1        NaN          NaN               NaN        NaN  ...   
     2000q2        NaN          NaN               NaN        NaN  ...   
     2000q3        NaN          NaN               NaN        NaN  ...   
     2000q4        NaN          NaN               NaN        NaN  ...   
     2001q1        NaN          NaN               NaN        NaN  ...   
...                ...          ...               ...        ...  ...   
ZUMZ 2021q2     9124.0    1017804.0          436023.0    67433.0  ...   
     2021q3     8770.0    1030742.0          436446.0    65844.0  ...   
     2021q4    10501.0     978189.0          446447.0    64389.0  ...   
     2022q1    12828.0     862012.0          393694.0    63577.0  ...   
     2022q2     1558.0     768114.0          391217.0    69327.0  ...   

             Net Income  Profit Margin      EPSP       SAG        OPG  \
firm q                                                                  
AAP  2000q1         NaN            NaN       NaN       NaN        NaN   
     2000q2         NaN            NaN       NaN       NaN        NaN   
     2000q3         NaN            NaN       NaN       NaN        NaN   
     2000q4         NaN            NaN       NaN       NaN        NaN   
     2001q1         NaN            NaN       NaN       NaN        NaN   
...                 ...            ...       ...       ...        ...   
ZUMZ 2021q2     26385.0       9.454651  0.027141  0.264552  67.973843   
     2021q3     23998.0       8.932280  0.031937       NaN        NaN   
     2021q4     30702.0      10.606830  0.036131       NaN        NaN   
     2022q1     38204.0      11.020056  0.067962       NaN        NaN   
     2022q2      -397.0      -0.179894  0.000982       NaN        NaN   

                  BMR       SFL       LFL         r       F1r  
firm q                                                         
AAP  2000q1       NaN       NaN       NaN       NaN       NaN  
     2000q2       NaN       NaN       NaN       NaN       NaN  
     2000q3       NaN       NaN       NaN       NaN       NaN  
     2000q4       NaN       NaN       NaN       NaN       NaN  
     2001q1       NaN       NaN       NaN       NaN       NaN  
...               ...       ...       ...       ...       ...  
ZUMZ 2021q2  0.460630  0.066253  0.234380  0.581803  0.357106  
     2021q3  0.593977  0.063880  0.216391  0.357106  0.266038  
     2021q4  0.482215  0.065825  0.225441  0.266038 -0.115775  
     2022q1  0.615212  0.073754  0.237014 -0.115775 -0.633520  
     2022q2  0.744932  0.090256  0.270732 -0.633520       NaN  

[20790 rows x 33 columns]

In [97]:
df_c = commercial.groupby(["firm"],as_index=False).take([-1])
df_c

,,,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,totalliabilities,shortdebt,...,Net Income,Profit Margin,EPSP,SAG,OPG,BMR,SFL,LFL,r,F1r
,firm,q,,,,,,,,,,,,,,,,,,,,,
0,AAP,2022q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12002481.0,9092801.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.277214,0.000000,0.293081,-0.144831,-5.286863
1,ABC,2022q2,60064600.0,58049232.0,1384266.0,143806.0,85649.0,52862.0,113120.0,57173630.0,56655766.0,1421566.0,...,550769.0,0.916961,0.021296,NaN,NaN,0.017475,0.024864,0.081159,0.225304,NaN
2,ABG,2022q2,3950200.0,3147500.0,466400.0,800.0,-28700.0,39100.0,66300.0,7638800.0,5228400.0,138400.0,...,202200.0,5.118728,0.089735,NaN,NaN,0.643165,0.018118,0.463869,-0.011916,NaN
3,ACI,2022q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28220000.0,24149400.0,1472400.0,...,NaN,NaN,NaN,NaN,NaN,0.300158,0.052176,0.447144,0.323732,NaN
4,AE,2022q2,992050.0,0.0,989090.0,0.0,0.0,-167.0,651.0,504623.0,337171.0,6536.0,...,2476.0,0.249584,0.021052,0.108736,-0.990852,1.190968,0.012952,0.025393,0.181133,-0.990261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,WSM,2022q2,1891227.0,1062679.0,505067.0,0.0,0.0,-163.0,69531.0,4210713.0,2900969.0,211614.0,...,254113.0,13.436409,0.042400,5.547822,-88.027441,0.171674,0.050256,0.246573,-0.346125,-3.447341
227,WSO,2022q2,2133755.0,1538222.0,314753.0,-6317.0,-32949.0,1110.0,60481.0,3788592.0,1568817.0,88600.0,...,186240.0,8.728275,0.030175,NaN,NaN,0.238555,0.023386,0.112746,-0.154603,NaN
228,WSTG,2022q2,67863.0,55377.0,8379.0,0.0,-442.0,7.0,867.0,179003.0,124590.0,999.0,...,2791.0,4.112698,0.027693,NaN,NaN,0.366899,0.005581,0.017374,0.304832,NaN


In [98]:
df_c["Market Value"]

     firm  q     
0    AAP   2022q2    1.049615e+07
1    ABC   2022q2    2.963497e+07
2    ABG   2022q2    3.747714e+06
3    ACI   2022q2    1.356151e+07
4    AE    2022q2    1.406016e+05
                         ...     
226  WSM   2022q2    7.629257e+06
227  WSO   2022q2    9.305085e+06
228  WSTG  2022q2    1.483051e+05
229  ZEUS  2022q2    2.865375e+05
230  ZUMZ  2022q2    5.059484e+05
Name: Market Value, Length: 231, dtype: float64

In [99]:
df_c_mv = df_c[["Market Value"]]
df_c_mv

,,,Market Value
,firm,q,
0,AAP,2022q2,1.049615e+07
1,ABC,2022q2,2.963497e+07
2,ABG,2022q2,3.747714e+06
3,ACI,2022q2,1.356151e+07
4,AE,2022q2,1.406016e+05
...,...,...,...
226,WSM,2022q2,7.629257e+06
227,WSO,2022q2,9.305085e+06
228,WSTG,2022q2,1.483051e+05


In [100]:
# Get size of each 
df_c_mv["Size"] = pd.cut(df_c_mv["Market Value"], bins=3, labels=["small", "medium", "big"])
df_c_mv

C:\Users\maria\AppData\Local\Temp\ipykernel_18212\3475481114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c_mv["Size"] = pd.cut(df_c_mv["Market Value"], bins=3, labels=["small", "medium", "big"])


,,,Market Value,Size
,firm,q,,
0,AAP,2022q2,1.049615e+07,small
1,ABC,2022q2,2.963497e+07,small
2,ABG,2022q2,3.747714e+06,small
3,ACI,2022q2,1.356151e+07,small
4,AE,2022q2,1.406016e+05,small
...,...,...,...,...
226,WSM,2022q2,7.629257e+06,small
227,WSO,2022q2,9.305085e+06,small
228,WSTG,2022q2,1.483051e+05,small


In [103]:
dummies = pd.get_dummies(df_c_mv["Size"], drop_first= True)
df_c_mv

,,,Market Value,Size
,firm,q,,
0,AAP,2022q2,1.049615e+07,small
1,ABC,2022q2,2.963497e+07,small
2,ABG,2022q2,3.747714e+06,small
3,ACI,2022q2,1.356151e+07,small
4,AE,2022q2,1.406016e+05,small
...,...,...,...,...
226,WSM,2022q2,7.629257e+06,small
227,WSO,2022q2,9.305085e+06,small
228,WSTG,2022q2,1.483051e+05,small


In [106]:
df_c_mv[["Medium"]] = dummies[["medium"]]
df_c_mv[["Big"]] = dummies[["big"]]
df_c_mv

C:\Users\maria\AppData\Local\Temp\ipykernel_18212\1286156951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c_mv[["Medium"]] = dummies[["medium"]]
C:\Users\maria\AppData\Local\Temp\ipykernel_18212\1286156951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c_mv[["Big"]] = dummies[["big"]]


,,,Market Value,Size,Medium,Big
,firm,q,,,,
0,AAP,2022q2,1.049615e+07,small,0,0
1,ABC,2022q2,2.963497e+07,small,0,0
2,ABG,2022q2,3.747714e+06,small,0,0
3,ACI,2022q2,1.356151e+07,small,0,0
4,AE,2022q2,1.406016e+05,small,0,0
...,...,...,...,...,...,...
226,WSM,2022q2,7.629257e+06,small,0,0
227,WSO,2022q2,9.305085e+06,small,0,0
228,WSTG,2022q2,1.483051e+05,small,0,0
